In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
from scipy.spatial.distance import cosine
from transformers import AutoModel, AutoTokenizer

In [ ]:
#data_train = pd.read_json("/content/drive/MyDrive/Deeeep learning/Sarcasm Headlines/train_Sarcasm_Headlines_Dataset.json", lines=True)
#data_test = pd.read_json("/content/drive/MyDrive/Deeeep learning/Sarcasm Headlines/test_Sarcasm_Headlines_Dataset.json", lines=True)
isarcasm_test = pd.read_csv("/content/drive/MyDrive/Deeeep learning/iSarcasm/isarcasm_test.csv")
gen_data = pd.read_csv('/content/drive/MyDrive/Deeeep learning/Sarcasm Headlines/sarcasm_headlines_synthetic__FULL__Llama_3_topp95_temp_7.csv')

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")
# model = AutoModel.from_pretrained("princeton-nlp/sup-simcse-bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Deeeep learning/models/RoBERTa-headlines-with-synt-50epochs")
model = AutoModel.from_pretrained("/content/drive/MyDrive/Deeeep learning/models/RoBERTa-headlines-with-synt-50epochs")

Some weights of RobertaModel were not initialized from the model checkpoint at /content/drive/MyDrive/Deeeep learning/models/RoBERTa-headlines-with-synt-50epochs and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# train_headlines = data_train['headline'].tolist()
# test_headlines = data_test['headline'].tolist()
train_headlines = gen_data['generated_sentence'].tolist()
test_headlines = isarcasm_test['text'].tolist()

In [ ]:
def create_embeddings(headlines, tokenizer, model):
    bert_embeddings = []
    i = 0
    while i < len(headlines):
        start = i
        end = min(i + 100, len(headlines))
        inputs = tokenizer(headlines[start:end], padding=True, truncation=True, return_tensors="pt").to(device)
        # Get the embeddings
        with torch.no_grad():
            embedding = model(**inputs, output_hidden_states=True, return_dict=True).pooler_output
        bert_embeddings.append(embedding)
        i += end - start
    bert_embeddings_tensor = torch.cat(bert_embeddings, dim=0)
    bert_embeddings_np = bert_embeddings_tensor.detach().cpu().numpy()
    return bert_embeddings_np

In [ ]:
%cd '/content/drive/MyDrive/Deeeep learning/embeddings'

/content/drive/MyDrive/Deeeep learning/embeddings


In [ ]:
train_embeddings_np = create_embeddings(train_headlines, tokenizer, model)
with open('synt_RoBERTa-headlines-with-synt-50epochs.npy', 'wb') as f:
    np.save(f, train_embeddings_np)

In [ ]:
test_embeddings_np = create_embeddings(test_headlines, tokenizer, model)
with open('isarcasm_RoBERTa-headlines-with-synt-50epochs.npy', 'wb') as f:
    np.save(f, test_embeddings_np)